In [ ]:
from __future__ import division, print_function

import os
import math

# Third-party
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from astropy.table import QTable

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
from scipy.optimize import minimize

from uncluster.conf import t_evolve
from uncluster.potential import potential as pot
from uncluster.cluster_massloss import solve_mass_radius

In [ ]:
gc_evo_filename = os.path.abspath("../output/2-gc-evolution.hdf5")

In [ ]:
with h5py.File(gc_evo_filename) as f:
    for i in range(f.attrs['n']):
        g = f['clusters/{}'.format(i)]

        t = f['time'][:g.attrs['disrupt_idx']+1]
        m = g['mass'][:]
        r = np.mean(g['radius'][:])
        ecc = g.attrs['ecc']
        break

In [ ]:
# choose a random 3D position at the mean orbital radius
phi = np.random.uniform(0, 2*np.pi)
theta = np.arccos(2*np.random.uniform() - 1)
q = np.vstack((r*np.cos(phi)*np.sin(theta),r*np.sin(phi)*np.sin(theta),r*np.cos(theta)))[:,0] * u.kpc

In [ ]:
dt = (2*np.pi*r*u.kpc / (200*u.km/u.s)).to(u.Myr) / 512

In [ ]:
def func(p):
    a,b = p
    v = gc.spherical_to_cartesian(q, [a, 0, b]*u.km/u.s)
    w0 = gd.CartesianPhaseSpacePosition(pos=q, vel=v)
    w = pot.integrate_orbit(w0, dt=dt, n_steps=1024)
    return (w.eccentricity() - ecc)**2

In [ ]:
res = minimize(func, [0,150], method='powell')
res

In [ ]:
v = gc.spherical_to_cartesian(q, [res.x[0], 0, res.x[1]]*u.km/u.s)
w0 = gd.CartesianPhaseSpacePosition(pos=q, vel=v)
w = pot.integrate_orbit(w0, dt=dt, n_steps=8192)
w.eccentricity()

In [ ]:
np.mean(np.sqrt(np.sum(w.pos**2, axis=0)))